In [1]:
import xarray as xr
import pandas as pd
import numpy as np
import ee
import geemap
import geetools
from datetime import datetime

In [2]:
ee.Initialize(project='ee-jonstar')

In [3]:
ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')

## CONUS Geometry

In [4]:
conus = ee.ImageCollection("GRIDMET/DROUGHT").filterDate('2000-01-01', '2001-01-01')
geometry = conus.geometry();

In [5]:
# Define point near Baltimore
u_lon = -76.6122
u_lat = 39.2904
u_poi = ee.Geometry.Point(u_lon, u_lat)
roi = u_poi.buffer(2e2)

## Landsat Initialization

In [6]:
landsat = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
landsat_US = landsat.filterDate('2024-01-01', '2024-09-01').filterBounds(roi)

In [7]:
landsat_US.first().projection()

In [8]:
lList = landsat_US.toList(8)

In [9]:
# List of days Landsat should have an image, noting missing ones
# January 14, 2024
# January 30, 2024 - MISSING
# February 15, 2024
# March 2, 2024 - MISSING
# March 18, 2024
# April 3, 2024 - MISSING
# April 19, 2024 - MISSING
# May 5, 2024 - MISSING
# May 21, 2024
# June 6, 2024
# June 22, 2024
# July 8, 2024
# July 24, 2024 - MISSING
# August 9, 2024 - MISSING
# August 25, 2024

ee.Image(lList.get(0)).date()

## GOES Initialization

In [10]:
# Landsat images over Baltimore regularly taken around 15:47 every 16 days
hour_filt = ee.Filter.calendarRange(15, 15, 'hour')
minute_filt = ee.Filter.calendarRange(45, 49, 'minute')

In [11]:
GOES = ee.ImageCollection("NOAA/GOES/16/MCMIPC").filterDate('2024-01-01', '2024-09-01')\
                .filter(hour_filt).filter(minute_filt)

In [12]:
gList = GOES.toList(15)

In [13]:
ee.Image(gList.get(13)).date()

## Filtering images by time

In [14]:
join = ee.Join.inner()
# Difference value is equal to 8 days in milliseconds
filterNear = ee.Filter.maxDifference(difference=691200000, leftField='system:time_start', rightField='system:time_start')

In [15]:
col_near = ee.ImageCollection(join.apply(landsat_US, GOES, filterNear))

In [16]:
col_near.size().getInfo()

124

In [17]:
landsat_US.size().getInfo()

8

In [18]:
GOES.size().getInfo()

240

## Helper Functions

In [19]:
# Applies scale and offset factors for Landsat band 10 LST
def scale_and_offset_LST(image):
    return image.select('ST_B10').multiply(0.00341802).add(149)

# Applies scale and offset factors for Landsat reflectance bands
def scale_and_offset_Landsat(image):
    return image.select(['SR_B2', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7']).multiply(2.75e-05).add(-0.2)

In [20]:
# Applies scale and offset factors for GOES imagery
# All scale and offset values were taken straight from GEE to save compute
def scale_and_offset_GOES(image, band):
    reflectances = image.select(['CMI_C01', 'CMI_C02', 'CMI_C03', 'CMI_C05', 'CMI_C06']).multiply(0.0002442)
    brightness_temps = image.select(['CMI_C14', 'CMI_C15']).multiply(0.039316241).add(173.14999)
    
    return reflectances.addBands(srcImg=brightness_temps, names=['CMI_C14', 'CMI_C15'])


def scale_and_offset_GOES2(image, band):
    if band == 14:
        scale = 0.059850749
        offset = 96.190002
        #print(scale, offset)
        return image.select('CMI_C14').multiply(scale).add(offset)
    elif band == 15:
        #scale = image.get('CMI_C15_scale').getInfo()
        scale = 0.05956082
        #offset = image.get('CMI_C15_offset').getInfo()
        offset = 97.379997
        #print(scale, offset)
        return image.select('CMI_C15').multiply(scale).add(offset)
    else:
        return image

In [21]:
# Preprocessing function
def process_feature(feature):
    #####################################
    # Landsat portion
    
    # Scaling and offset
    image1 = ee.Image(feature.get('primary'))
    landsat_LST = scale_and_offset_LST(image1)
    landsat_image = scale_and_offset_Landsat(image1)
    # Add cloud mask back in
    landsat_image = landsat_image.addBands(srcImg=landsat_LST, names=['ST_B10'])\
                                .addBands(srcImg=image1, names=['QA_PIXEL'])

    ######################################
    # GOES portion

    # Scaling and offset
    image2 = ee.Image(feature.get('secondary'))
    GOES_image = scale_and_offset_GOES(image2, 14)

    ########################################
    # Function to get median time of both images
    
    def get_GOES_time(f):
        GOES_time = ee.Number(image2.get('system:time_start'))       
        return f.set('timestamp', GOES_time)

    ##########################################
    # Add timestamp back in
    landsat_image = get_GOES_time(landsat_image)

    ##########################################
    
    image = landsat_image.addBands(srcImg=GOES_image, names=['CMI_C01', 'CMI_C02', 'CMI_C03', 'CMI_C05',
                                                             'CMI_C06', 'CMI_C14', 'CMI_C15']).cast({'QA_PIXEL':'double'})
    return image

## Produce geotiffs and their respective times

In [22]:
processed = col_near.map(process_feature)

In [25]:
first = processed.first()

In [23]:
times = processed.aggregate_array('timestamp')

In [24]:
def times_to_features(num):
    return ee.Feature(None, {'value': num})

features = ee.FeatureCollection(times.map(times_to_features))

In [27]:
task = ee.batch.Export.table.toDrive(
    collection=features,
    description='times',
    folder='GOES_Landsat_DMV',
    fileFormat='CSV',
)
task.start()

In [29]:
sample = ee.Image(processed.toList(3).get(2))

In [30]:
sample.get('timestamp')

In [25]:
# Export images, only taking area over Baltimore and Washington
num = processed.size().getInfo()
img_list = processed.toList(num)

for i in list(range(num)):
    img = ee.Image(img_list.get(i))
    
    task = ee.batch.Export.image.toDrive(
        img, description=f'GOES_Landsat_image_{i}', fileFormat='GeoTIFF', folder='GOES_Landsat_DMV',
                dimensions='3000x3000', crs='EPSG:32618', crsTransform=[30, 0, 292000, 0, -30, 4372200])
    task.start()

KeyboardInterrupt: 

In [27]:
num = processed.size().getInfo()
img_list = processed.toList(num)

for i in list(range(num)):
    img = ee.Image(img_list.get(i))
    
    task = ee.batch.Export.image.toDrive(
        img, description=f'GOES_Landsat_image_{i}', fileFormat='GeoTIFF', folder='GOES_Landsat_DMV',
                dimensions='7677x7908', crs='EPSG:32618', crsTransform=[30, 0, 220185, 0, -30, 4426815])
    task.start()

## Try from gee code

In [21]:
geo = sample.select('ST_B10').geometry()

In [22]:
Map = geemap.Map(center=[u_lat, u_lon], zoom=4)
Map.add_basemap("HYBRID")

Map.addLayer(sample.select('ST_B10'), {'min':250, 'max':325,
    'palette':['#fef0d9', '#fdcc8a', '#fc8d59', '#e34a33', '#b30000']}, 'LST')
Map.addLayer(sample.select('CMI_C14').clip(geo), {'min':96, 'max':342,
    'palette':['#f7f7f7', '#cccccc', '#969696', '#636363', '#252525']}, 'GOES14')

In [23]:
Map

Map(center=[39.2904, -76.6122], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchD…